In [1]:
include("../src/Julia.jl")

objc[29891]: Class TKApplication is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[29891]: Class TKMenu is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[29891]: Class TKContentView is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[29891]: Class TKWindow is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.


pcgWorker (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
gpath = "../../graphs/";

In [4]:
graphs = readdir("../../graphs");

In [5]:
for cur in graphs
    path = gpath * cur * "/"
    
    try
        a = readFromFile(path * "graph.mtx");
        x = readFromFile(path * "x.vec");
        
        tree = []
        for ind in 1:5
            tr = readFromFile(path * "tree$(ind).mtx");
            
            u,v,w = findnz(tr)
            for i in 1:length(w)
                w[i] = a[u[i],v[i]]
            end
            tr = sparse(u,v,w)
            
            push!(tree, tr);
        end
        
        la = lap(a);
        x = x - mean(x)
        b = la * x; b = b - mean(b);
        
        numIts = 200;
        @time for i in 4:5
            @time F = treeSolver(tree[i])
            @time F(b);

            # a run on standard Float64 data types
            @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
            norm(lap(a) * myx64 - b) / norm(b)
            insert!(dbg, 1, "$(numIts) iterations")
            insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
            writeToFile(path * "_log_julia_tree$(i)_default.txt", dbg)

            # now useing higer precision data types
            for hp in [64,128,256,512,1024]
                set_bigfloat_precision(hp)
                laHP = toHighPrecision(la; precision=hp)
                treeHP = toHighPrecision(tree[i]; precision=hp)
                xHP = toHighPrecision(x; precision=hp);
                bHP = toHighPrecision(b; precision=hp);
                FHP = treeSolver(treeHP)

                @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=false);
                insert!(dbg, 1, "$(numIts) iterations")
                insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
                writeToFile(path * "_log_julia_tree$(i)_$(hp).txt", dbg)
            end
        end
            
        println("finished running on ", cur)
    catch
        println("skipping ", cur)
    end
end

  0.422706 seconds (652.01 k allocations: 30.341 MB, 1.84% gc time)
  0.135419 seconds (290.67 k allocations: 7.535 MB)
  1.638473 seconds (39.95 M allocations: 968.070 MB, 3.96% gc time)
 79.650361 seconds (476.35 M allocations: 12.112 GB, 31.83% gc time)
 90.031878 seconds (476.14 M allocations: 13.262 GB, 32.56% gc time)
 93.867059 seconds (476.14 M allocations: 15.579 GB, 31.82% gc time)
102.651312 seconds (476.14 M allocations: 20.214 GB, 32.08% gc time)
115.645276 seconds (476.14 M allocations: 29.484 GB, 30.32% gc time)
  0.015418 seconds (50.18 k allocations: 5.763 MB)
  0.005597 seconds (197.90 k allocations: 3.631 MB)
  1.736908 seconds (39.79 M allocations: 959.265 MB, 24.40% gc time)
 79.937883 seconds (476.19 M allocations: 12.104 GB, 32.45% gc time)
 86.481760 seconds (476.19 M allocations: 13.263 GB, 33.18% gc time)
 95.796348 seconds (476.19 M allocations: 15.580 GB, 31.75% gc time)
106.120554 seconds (476.19 M allocations: 20.215 GB, 32.01% gc time)
116.862482 seconds 